In [1]:
'''
Getting the ABDU model to work in notebook

EPSG: 5070
'''
import duckdb
import geopandas as gpd
import leafmap
from shapely import wkt
import pandas as pd
import pyarrow as pa
import geoarrow.pyarrow as ga
import geoarrow.pandas as _
import rasterio
from rasterio import mask
from shapely.geometry import shape

con = duckdb.connect()
con.install_extension("spatial")
con.load_extension("spatial")
con.install_extension("azure")
con.load_extension("azure")
con.install_extension("json")
con.load_extension("json")

In [2]:
gpd.__version__

'0.14.1'

In [3]:
'''
Read in AOI.  Need it projected as 4326 and 5070.  Unfortunately projecting isn't working smoothly for me and keeps providing infinite values.
'''
inaoifile = 'test.parquet'
#con.sql("CREATE TABLE aoi AS SELECT * FROM ST_READ('{0}')".format(inaoi))
inaoigpd = gpd.read_parquet(inaoifile)
inepsg = inaoigpd.crs.to_epsg()
inaoi = pd.read_parquet(inaoifile)
if inepsg == 5070:
    con.sql("CREATE OR REPLACE TABLE aoi5070 AS SELECT * EXCLUDE geometry, ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM inaoi")
    inaoi4326 = inaoigpd.to_crs(4326)
    inaoi4326['geometry'] = inaoi4326.to_wkb().geometry
    inaoi4326 = pd.DataFrame(inaoi4326)
    con.sql("CREATE OR REPLACE TABLE aoi4326 AS SELECT * EXCLUDE geometry, ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM inaoi4326")
elif inepsg == 4326:
    con.sql("CREATE OR REPLACE TABLE aoi4326 AS SELECT * EXCLUDE geometry, ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM inaoi")
    inaoi5070 = inaoi.to_crs(5070)
    inaoi5070['geometry'] = inaoi5070.to_wkb().geometry
    inaoi5070 = pd.DataFrame(inaoi5070)
    con.sql("CREATE OR REPLACE TABLE aoi5070 AS SELECT * EXCLUDE geometry, ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM inaoi5070")
else:
    inaoi5070 = inaoi.to_crs(5070)
    inaoi5070['geometry'] = inaoi5070.to_wkb().geometry
    inaoi5070 = pd.DataFrame(inaoi5070)
    con.sql("CREATE OR REPLACE TABLE aoi5070 AS SELECT * EXCLUDE geometry, ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM inaoi5070")
    inaoi4326 = inaoi.to_crs(4326)
    inaoi4326['geometry'] = inaoi4326.to_wkb().geometry
    inaoi4326 = pd.DataFrame(inaoi4326)    
    con.sql("CREATE OR REPLACE TABLE aoi4326 AS SELECT * EXCLUDE geometry, ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM inaoi4326")

In [4]:
'''
Read in hucs partitioned to huc2/huc4 level that overlap with the aoi.  Don't clip hucs
'''
con.sql("SET azure_storage_connection_string = 'DefaultEndpointsProtocol=https;AccountName=giscog;EndpointSuffix=core.windows.net';")
con.sql(f"""
CREATE OR REPLACE TABLE huc12 AS
SELECT LEFT(huc12,2) AS huc2,LEFT(huc12,4) AS huc4, huc12, areaacres, huc.geometry
FROM (SELECT huc12, areaacres, geometry FROM read_parquet('azure://abdu/huc/**/*.parquet')
WHERE CAST(LEFT(huc12,2) AS INTEGER)<=12) AS huc
JOIN aoi5070 ON 
ST_Intersects(ST_GeomFromWKB(huc.geometry), ST_GeomFromText(aoi5070.geometry))
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
%%time
hucs = con.sql("select huc4 from huc12 GROUP BY huc4").df().values.tolist()
hucs = sorted([item for items in hucs for item in items])
print(hucs)
'''
Clip wetlands that aren't riverine to the hucs selected by the aoi.
'''
con.sql("""
CREATE OR REPLACE TABLE wetlands AS
SELECT ATTRIBUTE, huc12.geometry
FROM (SELECT ATTRIBUTE, geometry FROM read_parquet('azure://abdu/nwi/**/*.parquet')
WHERE huc4 IN {0} AND WETLAND_TYPE != 'Riverine') AS wetlnd
JOIN huc12 ON 
ST_Intersects(ST_GeomFromWKB(wetlnd.geometry), ST_GeomFromWKB(huc12.geometry))
""".format(tuple(hucs)))

['0801', '0802']


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 1min 46s
Wall time: 8min 28s


In [6]:
%%time
'''
Clip wetlands that aren't riverine to the hucs selected by the aoi.
'''
selectstring = """CREATE OR REPLACE TABLE wetlands AS
SELECT ATTRIBUTE,huc2, huc4, ST_AsText(ST_Intersection(ST_GeomFromWKB(wetlnd.geometry), ST_GeomFromWKB(huc12.geometry))) as geometry
FROM (SELECT ATTRIBUTE, geometry FROM read_parquet('azure://abdu/nwi/**/*.parquet') 
WHERE WETLAND_TYPE != 'Riverine' AND huc4 IN {0}) AS wetlnd
JOIN huc12 ON 
ST_Intersects(ST_GeomFromWKB(wetlnd.geometry), ST_GeomFromWKB(huc12.geometry))
""".format(tuple(hucs))
con.sql(selectstring)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: total: 1min 47s
Wall time: 8min 7s


In [7]:
'''
Read in demand clipped by hucs
'''
con.sql(f"""
CREATE OR REPLACE TABLE demandfull AS SELECT * EXCLUDE geometry, ST_AsText(ST_GeomFromWKB(read_parquet.geometry)) as geometry
FROM read_parquet('azure://abdu/Demand9Species.parquet')
JOIN huc12 ON 
ST_Intersects(ST_GeomFromWKB(read_parquet.geometry), ST_GeomFromWKB(huc12.geometry))
""")
con.sql(f"""
CREATE OR REPLACE TABLE demand AS SELECT * EXCLUDE geometry, ST_AsText(ST_Intersection(ST_GeomFromWKB(huc12.geometry), ST_GeomFromWKB(read_parquet.geometry))) as geometry
FROM read_parquet('azure://abdu/Demand9Species.parquet')
JOIN huc12 ON 
ST_Intersects(ST_GeomFromWKB(huc12.geometry), ST_GeomFromWKB(read_parquet.geometry))
""")
con.sql("""CREATE OR REPLACE TABLE demand AS SELECT fips, huc12, CODE, LTADUD, LTADemand, LTAPopObj, x80DUD, X80Demand, X80PopObj, ST_Area(geometry)*0.0001 AS ha, geometry FROM (
SELECT fips, huc12.huc12, CODE, LTADUD, LTADemand, LTAPopObj, x80DUD, X80Demand, X80PopObj, ST_Intersection(ST_GeomFromText(demand.geometry), ST_GeomFromWKB(huc12.geometry)) as geometry FROM demand
JOIN huc12 ON ST_Intersects(ST_GeomFromText(demand.geometry), ST_GeomFromWKB(huc12.geometry))
WHERE species='All'
)
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
'''
Read in PADUS
'''
con.sql("""
CREATE OR REPLACE TABLE protected AS 
SELECT CATEGORY, huc2, huc4, ST_AsText(ST_Intersection(ST_GeomFromWKB(huc12.geometry), ST_GeomFromWKB(prot.geometry))) as geometry
FROM (SELECT CATEGORY, geometry FROM read_parquet('azure://abdu/padus/**/*.parquet')
WHERE CATEGORY IN ('Fee', 'Easements', 'Other') AND huc4 IN {0}) AS prot
JOIN huc12 ON 
ST_Intersects(ST_GeomFromWKB(huc12.geometry), ST_GeomFromWKB(prot.geometry))
""".format(tuple(hucs)))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [9]:
'''
Read in NLCD clipped to hucs
'''
with rasterio.open('https://giscog.blob.core.windows.net/newcontainer/nlcd2019_cog.tif') as src:
    # Clip the raster to the geometry of the shapefile
    clipped_data, transform = mask.mask(src, inaoigpd.geometry, crop=True)

clipped_data[clipped_data>23]=0
clipped_data[clipped_data<21]=0
clipped_data[clipped_data==21]=1
clipped_data[clipped_data==22]=1
clipped_data[clipped_data==23]=1
shapes = rasterio.features.shapes(clipped_data[0], transform=transform, mask=clipped_data[0] == 1)
# Create a GeoDataFrame from the vector polygons
gdf_vector = gpd.GeoDataFrame({'geometry': [shape(geom) for geom, value in shapes]})
gdf_vector['geometry'] = gdf_vector.to_wkb().geometry
con.sql("CREATE OR REPLACE TABLE urban AS SELECT * EXCLUDE geometry, ST_AsText(ST_GeomFromWKB(geometry)) AS geometry FROM gdf_vector")

In [10]:
'''
Import wetland crossclass data and assign classes to the nwi table
'''
con.sql("""CREATE OR REPLACE TABLE crossnwi AS (UNPIVOT (FROM (SELECT * FROM read_json_auto('aoiWetland.json', maximum_object_size=100000000))) ON COLUMNS(*))""")
con.sql("""CREATE OR REPLACE TABLE crossnwi AS SELECT name, UNNEST(value) AS value FROM crossnwi""")
con.sql("""CREATE OR REPLACE TABLE wetlands AS
SELECT name, geometry FROM (SELECT * FROM wetlands) AS wetselect
LEFT JOIN crossnwi ON wetselect.ATTRIBUTE LIKE crossnwi.value
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
'''
#################################
End of data import
Starting model process
#################################
'''
#### Prepping energy - Join energy to nwi.  Need to create the spatial kcal table first. What's the best way to do this?
# parquet is the best to read in but it's not easily editable.  Rest service would be ok but again, not great because
# reading those is difficult.  I wonder if

'\n#################################\nEnd of data import\nStarting model process\n#################################\n'

In [12]:
'''
Demand

######
Need to proportion demand based on available energy.  Available energy is spatially explicit but demand is at the fips
count level.  We need to calculate total energy and demand at the huc12 scale.
To proportion demand we need to calclulate total energy by fips then calculate how much energy is in each huc12. A proportion
can then be calculated by dividing total energy within a fips by (huc12,fips) group.  Demand at the huc12 level is multiplied
by that energy proportion.
######
'''

'\nDemand\n\n######\nNeed to proportion demand based on available energy.  Available energy is spatially explicit but demand is at the fips\ncount level.  We need to calculate total energy and demand at the huc12 scale.\nTo proportion demand we need to calclulate total energy by fips then calculate how much energy is in each huc12. A proportion\ncan then be calculated by dividing total energy within a fips by (huc12,fips) group.  Demand at the huc12 level is multiplied\nby that energy proportion.\n######\n'

In [13]:
# Need to sum energy within fips
# The aoi touches multiple huc12s which intersect multiple fips
con.sql("""
CREATE OR REPLACE TABLE demandfull AS SELECT DISTINCT fips, ST_Union_Agg(ST_GeomFromText(geometry)) as geometry from demandfull
group by fips
""")
fullfips = con.sql("""SELECT fips from demandfull""").df().values.tolist()
fullfips

[['29069'], ['05093'], ['47097'], ['29155'], ['05111']]

In [14]:
'''
Iterate through fips.
Clip wetlands that aren't riverine to the fips selected by the aoi to calculate total energy within the fips
'''
tmp = pd.DataFrame()
for fip in fullfips:
    selectstring = """
    CREATE OR REPLACE table tmpwet AS
    (
    SELECT ATTRIBUTE,fips, ST_AsText(ST_Intersection(ST_GeomFromWKB(wetlnd.geometry), dmd.geometry)) as geometry
    FROM (SELECT ATTRIBUTE, geometry FROM read_parquet('azure://abdu/nwi/**/*.parquet')
    WHERE WETLAND_TYPE != 'Riverine') AS wetlnd
    JOIN (SELECT fips, geometry FROM demandfull WHERE fips='{0}') as dmd ON 
    ST_Intersects(ST_GeomFromWKB(wetlnd.geometry), dmd.geometry)
    )
    """.format(fip[0])
    con.sql(selectstring)
    con.sql("""CREATE OR REPLACE TABLE crossnwi AS (UNPIVOT (FROM (SELECT * FROM read_json_auto('aoiWetland.json', maximum_object_size=100000000))) ON COLUMNS(*))""")
    con.sql("""CREATE OR REPLACE TABLE crossnwi AS SELECT name, UNNEST(value) AS value FROM crossnwi""")
    con.sql("""CREATE OR REPLACE TABLE tmpwet AS
    SELECT name, fips, geometry FROM (SELECT * FROM tmpwet) AS wetselect
    LEFT JOIN crossnwi ON wetselect.ATTRIBUTE LIKE crossnwi.value
    """)
    con.sql(f"""CREATE OR REPLACE TABLE tmpwet AS
    (SELECT replace(tmpwet.name, '_', '') AS name, fips, geometry, kcal FROM tmpwet
    LEFT JOIN read_csv_auto('azure://abdu/kcal.csv') ON replace(tmpwet.name, '_', '') = read_csv_auto.habitatType
    WHERE tmpwet.name IS NOT NULL)
    """)
    con.sql("""CREATE OR REPLACE TABLE newdata AS (
    SELECT name, fips, kcal, ST_Area(geometry)*0.0001 AS ha, ha*kcal AS avalNrgy FROM(
    SELECT name, fips, kcal, ST_Union_Agg(ST_GeomFromText(geometry)) AS geometry FROM tmpwet
    GROUP BY name, fips, kcal))
    """)
    final = con.sql("SELECT fips, SUM(avalNrgy) from newdata GROUP BY fips").df()
    tmp = pd.concat([tmp, final])
con.sql("""CREATE OR REPLACE TABLE fipsavalNRG AS SELECT fips, "sum(avalNrgy)" as fipsnrgysum from tmp""")
con.sql("""select * from fipsavalNRG""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────┬────────────────────┐
│  fips   │    fipsnrgysum     │
│ varchar │       double       │
├─────────┼────────────────────┤
│ 29069   │  2179175072.302092 │
│ 05093   │  4343586557.220324 │
│ 47097   │  6412826086.457127 │
│ 29155   │ 3659891166.0527077 │
│ 05111   │  4853062003.605463 │
└─────────┴────────────────────┘

In [15]:
'''
######
Read in kcal.csv from azure and join to wetlands.
This file is the habitat type (habitatType) and the kcal/Ha (kcal)
######
'''
#
con.sql(f"""CREATE OR REPLACE TABLE wetlands AS
(SELECT replace(wetlands.name, '_', '') AS name, geometry, kcal FROM wetlands
LEFT JOIN read_csv_auto('azure://abdu/kcal.csv') ON replace(wetlands.name, '_', '') = read_csv_auto.habitatType
WHERE wetlands.name IS NOT NULL)
""")

In [16]:
con.sql("""CREATE OR REPLACE TABLE hucdemandenergy AS 
    (SELECT name,fips, huc12, CODE, LTADUD, LTADemand, LTAPopObj, x80DUD, X80Demand, X80PopObj, kcal, 
    ST_Intersection(ST_GeomFromText(wetlands.geometry), (demand.geometry)) as geometry FROM wetlands
    JOIN demand ON ST_Intersects(ST_GeomFromText(wetlands.geometry), (demand.geometry)))""")

In [17]:
'''
######
Calculate available energy (avalNrgy) of wetlands by calculating area in Hectares (HA) and multiplying by kcal.
Select only distinct rows.
Create new table habitatenergy
######
'''
#
con.sql("""CREATE OR REPLACE TABLE hucdemandenergy AS (SELECT DISTINCT name, fips, huc12, CODE, LTADUD, LTADemand, LTAPopObj, x80DUD, X80Demand, X80PopObj, kcal, geometry, ST_Area(geometry)*0.0001 AS ha,ha*kcal AS avalNrgy FROM hucdemandenergy)""")
# DATA CHECK
# con.sql("""COPY (SELECT DISTINCT name, ST_AsWKB(ST_GeomFromText(geometry)) as geometry, ST_Area(ST_GeomFromText(geometry))*0.0001 AS ha, ha*kcal AS avalNrgy FROM wetlands) TO 'testwetland.parquet' (FORMAT PARQUET)""")

In [18]:
con.sql("""CREATE OR REPLACE TABLE test AS (select name, hucdemandenergy.fips as fips,huc12, CODE, LTADUD, LTADemand, LTAPopObj, x80DUD, X80Demand, X80PopObj, kcal, avalNrgy, fipsnrgysum, (hucdemandenergy.avalNrgy/fipsavalNRG.fipsnrgysum) as pct, geometry from hucdemandenergy
    JOIN fipsavalNRG on hucdemandenergy.fips = fipsavalNRG.fips
    GROUP BY name, hucdemandenergy.fips, huc12, CODE, LTADUD, LTADemand, LTAPopObj, x80DUD, X80Demand, X80PopObj, kcal, avalNrgy, fipsnrgysum, geometry)""")

In [19]:
fullfips = sorted([item for items in fullfips for item in items])
sqlcall ="""CREATE OR REPLACE TABLE rdydemand as SELECT * FROM test WHERE fips In {0}""".format(tuple(fullfips))
con.sql(sqlcall)

In [24]:
#con.sql("""describe rdydemand""")
con.sql("""CREATE OR REPLACE TABLE hucdemand AS (SELECT huc12, code, 
sum(pct * LTADUD) AS LTADUD,
sum(pct * LTADemand) AS LTADemand,
sum(pct * LTAPopObj) AS LTAPopObj,
sum(pct * x80DUD) AS x80DUD,
sum(pct * X80Demand) AS X80Demand,
sum(pct * X80PopObj) AS X80PopObj,
geometry
FROM rdydemand
GROUP BY huc12, code, geometry)""")

BinderException: Binder Error: Referenced column "pct" not found in FROM clause!
Candidate bindings: "rdydemand.CODE"
LINE 2: sum(pct * LTADUD) AS LTADUD,
            ^

In [ ]:
'''
END Demand
'''